# MODEL Evaluation: Using Validation Set

In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

region = boto3.Session().region_name

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")

sess = sagemaker.Session()

# project bucket
bucket_name = "aai-540-group4"

# specifiy output location of training data and model
model_prefix = "sg-ic-transfer-learning"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
arn:aws:iam::324183265896:role/service-role/AmazonSageMaker-ExecutionRole-20250604T045982


In [2]:
import pandas as pd
import json

In [3]:
# load validation predictions 
s3_client.download_file(
    bucket_name, "sg-ic-transfer-learning/predictions/update-preds-25751.json", "predictions.json"
)

In [4]:
# load json file and convert to df
with open('predictions.json', 'r') as file:  # Replace with your file name if different
    predicted_labels = json.load(file)

In [5]:
pred_labels_df = pd.DataFrame(predicted_labels)
pred_labels_df.head()

,filename,pred_label
0,5858bf2c-23d2-11e8-a6a3-ec086b02610b_0.jpg,raccoon
1,5858bf4a-23d2-11e8-a6a3-ec086b02610b_0.jpg,raccoon
2,5858bf5d-23d2-11e8-a6a3-ec086b02610b_0.jpg,raccoon
3,5858bf6a-23d2-11e8-a6a3-ec086b02610b_0.jpg,raccoon
4,5858bf6d-23d2-11e8-a6a3-ec086b02610b_0.jpg,dog


In [6]:
# load grp4-metedata-val.json
with open('grp4-metadata-val.json', 'r') as file:  # Replace with your file name if different
    true_labels = json.load(file)

In [7]:
true_labels_df = pd.DataFrame(true_labels)

In [8]:
true_labels_df.head()

,filename,label,category_id,bbox,image_id,location,split_type
0,59fae563-23d2-11e8-a6a3-ec086b02610b_0.jpg,rabbit,10,"[1456.0, 480.0, 156.8, 137.6]",59fae563-23d2-11e8-a6a3-ec086b02610b,46,val
1,595f7b9c-23d2-11e8-a6a3-ec086b02610b_0.jpg,rodent,99,"[249.3516082764, 798.4633789062, 291.603164672...",595f7b9c-23d2-11e8-a6a3-ec086b02610b,67,val
2,5883e97b-23d2-11e8-a6a3-ec086b02610b_0.jpg,opossum,1,"[94.72, 1231.36, 655.36, 250.88]",5883e97b-23d2-11e8-a6a3-ec086b02610b,78,val
3,58d47cf1-23d2-11e8-a6a3-ec086b02610b_0.jpg,opossum,1,"[729.6, 1142.613359375, 660.48, 280.746640625]",58d47cf1-23d2-11e8-a6a3-ec086b02610b,100,val
4,588a66f6-23d2-11e8-a6a3-ec086b02610b_0.jpg,raccoon,3,"[540.16, 1013.76, 394.24, 480.0]",588a66f6-23d2-11e8-a6a3-ec086b02610b,100,val


In [9]:
# merge the two dfs on filename
merged_df = pd.merge(true_labels_df, pred_labels_df, on='filename')

In [10]:
len(merged_df)

25751

In [11]:
merged_df.loc[merged_df.isnull().any(axis=1)]

,filename,label,category_id,bbox,image_id,location,split_type,pred_label


In [12]:
# generate classification report
from sklearn.metrics import classification_report

# Suppose your DataFrame is named df
# 'true_label' is the column with actual values
# 'pred_label' is the column with predicted values

print(classification_report(merged_df['label'], merged_df['pred_label']))

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


               precision    recall  f1-score   support

       badger       1.00      0.17      0.29        18
         bird       0.76      0.84      0.80      1291
       bobcat       0.87      0.76      0.81      2844
          car       1.00      0.99      0.99      1066
          cat       0.65      0.57      0.61      1392
       coyote       0.75      0.83      0.79      2293
         deer       0.90      0.78      0.84      3017
          dog       0.74      0.83      0.78      1010
        empty       0.00      0.00      0.00        23
          fox       0.58      0.60      0.59       428
       lizard       0.31      0.14      0.19       123
mountain_lion       0.00      0.00      0.00         0
      opossum       0.92      0.94      0.93      4972
       rabbit       0.62      0.85      0.72       753
      raccoon       0.94      0.83      0.88      4832
       rodent       0.78      0.77      0.77       424
        skunk       0.76      0.91      0.83       469
     squi

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
